In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from tqdm import tqdm

import amelia_datatools.utils.common as C

out_dir = f"{C.OUTPUT_DIR}/count"
os.makedirs(out_dir, exist_ok=True)


In [ ]:
airport = "katl"
base_dir = C.DATA_DIR
traj_version = C.VERSION

traj_dir = os.path.join(base_dir, f'traj_data_{traj_version}', 'raw_trajectories', f'{airport}')

traj_files = [os.path.join(traj_dir, f) for f in os.listdir(traj_dir)]
print(traj_files[0])

data = pd.read_csv(traj_files[0])
data.head()

/Users/alonso.cano/Developer/CMU/Amelia/AmeliaDataTools/datasets/amelia/traj_data_a10v08/raw_trajectories/katl/KATL_498_1682107200.csv


,Frame,ID,Altitude,Speed,Heading,Lat,Lon,Range,Bearing,Type,Interp,x,y
0,0,17335176,1006.25,16.666667,270.582275,33.64849,-84.437928,1.606820,-0.619965,2.0,[ORG],1.307789,-0.933573
1,0,17336725,1006.25,0.000000,0.000000,33.64047,-84.420500,0.801022,1.021473,2.0,[ORG],0.418222,0.683174
2,0,17336862,1006.25,32.000000,69.713745,33.63601,-84.439076,1.043019,-1.644141,2.0,[ORG],-0.076431,-1.040215
3,0,17336755,1016.25,112.500000,90.800903,33.64954,-84.434894,1.566424,-0.429385,0.0,[ORG],1.424227,-0.652120
4,0,17336885,1006.25,0.000000,9.997559,33.64857,-84.438944,1.670340,-0.662811,2.0,[ORG],1.316672,-1.027820


In [11]:
agent_counts = {
    'total': [], '0': [], '1': [], '2': []
}
total_count = 0
for f in tqdm(traj_files):
    data = pd.read_csv(f)

    unique_frames = np.unique(data.Frame)
    for frame in unique_frames:
        agents = data[data.Frame == frame]

        agent_counts['total'].append(agents.shape[0])
        agent_counts['0'].append(np.where(agents.Type == 0.0)[0].shape[0])
        agent_counts['1'].append(np.where(agents.Type == 1.0)[0].shape[0])
        agent_counts['2'].append(np.where(agents.Type == 2.0)[0].shape[0])


100%|██████████| 686/686 [19:07<00:00,  1.67s/it]


In [12]:
agent_count_stats = {}
for k, v in agent_counts.items():
    v = np.asarray(v)
    agent_count_stats[k] = {
        "min": round(v.min().astype(float), 5),
        "max": round(v.max().astype(float), 5),
        "mean": round(v.mean().astype(float), 5),
        "std": round(v.std().astype(float), 5)
    }

    agent_counts[k] = v

agent_count_stats

{'total': {'min': 1.0, 'max': 115.0, 'mean': 43.51777, 'std': 16.89383},
 '0': {'min': 0.0, 'max': 64.0, 'mean': 13.04162, 'std': 10.0276},
 '1': {'min': 0.0, 'max': 14.0, 'mean': 2.41698, 'std': 1.84872},
 '2': {'min': 0.0, 'max': 75.0, 'mean': 28.05275, 'std': 10.13574}}

In [13]:
with open(f'{out_dir}/{airport}_stats.json', 'w') as f:
    json.dump(agent_count_stats, f, indent=2)

with open(f'{out_dir}/{airport}.pkl', 'wb') as handle:
    pickle.dump(agent_counts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
name = {'total': 'Total', '0': 'Aircraft', '1': 'Vehicle', '2': 'Unknown'}
colors = {'ksea': 'blue', 'kewr': 'red', 'kbos': 'green', 'kmdw': 'orange', 'katl': 'purple'}
for key, value in name.items():
    arr = agent_counts[key]

    plt.hist(arr, bins = int(arr.max()), color=colors[airport])
    plt.title(f"{value}")
    plt.xlabel("Number of Agents per Timestep")
    plt.ylabel("Count")
    plt.savefig(f"{out_dir}/{airport}_{key}.png", dpi=600)
    plt.close()

In [15]:
for key, value in name.items():
    arr = agent_counts[key]
    q_lower = np.quantile(arr, 0.05)
    q_upper = np.quantile(arr, 0.95)

    arr = arr[(arr >= q_lower) & (arr <= q_upper)]

    plt.hist(arr, bins = int(arr.max()), color=colors[airport])
    plt.title(f"{value}")
    plt.xlabel("Number of Agents per Timestep")
    plt.ylabel("Count")
    plt.savefig(f"{out_dir}/{airport}_{key}_iqr.png", dpi=600)
    plt.close()